In [1]:
import torch
from torch.utils.data import DataLoader
import monai 
from monai.transforms import (
    AddChanneld,
    Compose,
    LoadImaged,
    Invertd,
    Resized,
    EnsureTyped,
    SqueezeDimd,
    AddChannel,
    EnsureType
)
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.functional as F
from sklearn.model_selection import train_test_split
import torch._tensor
from pathlib import Path
from monai.visualize import plot_2d_or_3d_image
import torchvision.models as models
import segmentation_models_pytorch as smp
import pytorch_lightning as pl 
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import wandb 
from torchmetrics.functional import accuracy, dice_score, average_precision, matthews_corrcoef, precision_recall, jaccard_index
from torchmetrics import AveragePrecision, AUROC
import pydicom as dicom
import nibabel as nib
import glob
import wandb 
from pathlib import Path 
from SarcoDataModule import SarcoDataModule
#from SarcoInfDataModule import SarcoInfDataModule
from manual_cropping import *
from data_augment import *
from monai.transforms import MapTransform, InvertibleTransform, Resize
import itertools
from itertools import product
import dicom2nifti as d2n 

ModuleNotFoundError: No module named 'segmentation_models_pytorch'

In [2]:
#defining some metrics 

SMOOTH = 1e-6

def iou_and_dice_pytorch(output: torch.Tensor, label: torch.Tensor):

    outputs = torch.flatten(torch.squeeze(output, dim=1))  # BATCH x 1 x H x W => BATCH x H x W
    labels = torch.flatten(torch.squeeze(label,dim=1))
    
    intersection = abs(float(outputs.dot(labels)))

    union = abs(float(torch.sum(labels))+ float(torch.sum(outputs)) - intersection  )  
    sum_pixels= torch.sum(outputs) + torch.sum(labels)
    
    iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0
    dice= 2*(intersection/sum_pixels)
        
    return iou, dice 

In [5]:
#try collect directly nii files and dicom files
dirpath_raw=Path('/home/ghita/milvuetap/ghita_exploration/data/sarco dataset 1 & 2 improved')
problem_path='blabla'
files_raw=[filename for filename in dirpath_raw.glob('*') if filename != problem_path]
data=[]
for k in range(0,len(sorted(files_raw)),2):
    input_dict=dirpath_raw/ Path(sorted(files_raw)[k])
    label_dict=dirpath_raw/ Path(sorted(files_raw)[k+1])
    data.append({"input":input_dict ,"label":label_dict})

data

transform=LoadImaged(keys=['input', 'label'], dtype='float32')
ex_ds=monai.data.Dataset(data=data, transform=transform)

AttributeError: 'NoneType' object has no attribute 'summary'

In [13]:
#collecting data and splitting between train and val 
dirpath=Path('data/sarco_final1')
problem_path=Path('/home/ghita/milvuetap/ghita_exploration/data/sarco_final1/HyuyQN0Tca')
files=[filename for filename in dirpath.glob('*') if filename != problem_path]
print(len(files))
data=[]
for k in range(len(sorted(files))):
    filename=sorted(files)[k].name.replace('.dcm', '')
    input_dict=dirpath/filename/f"{filename}.dcm.npy"
    label_dict=dirpath/filename/f"{filename}.nii.npy"
    data.append({"input":input_dict ,"label":label_dict})


train_files, val_files=train_test_split(data,test_size=0.2)



341


In [2]:
transforms=Compose(
    [LoadImaged(keys=["input"], reader='numpyreader', dtype='float32', image_only=True),
    LoadImaged(keys=["label"], reader='numpyreader', dtype='float32', image_only=True),
    SqueezeDimd(keys=['label'],dim=2),
    AddChanneld(keys=["input","label" ]),
    Resized(keys=["input"],spatial_size=(512,512)),
    Resized(keys=["label"], spatial_size=(512,512))
    ]
)  

In [26]:

from monai.transforms import (
    AddChanneld,
    Compose,
    EnsureTyped,
    LoadImaged,
    RandRotate90d,
    Resized,
    SqueezeDimd,
)
from utils.data_augment import ProductDataset, Mixupd

ex_dataset=monai.data.Dataset(data=train_files, transform=transforms)
prod=ProductDataset(ex_dataset, ex_dataset, transform=Compose(
                        [
                            Mixupd(keys=["input", "label"], alpha=1),
                            RandRotate90d(
                                keys=["input", "label"], prob=0.5, spatial_axes=[0, 1]
                            ),
                            EnsureTyped(
                                keys=["input"], data_type="tensor", dtype=torch.float
                            ),
                            EnsureTyped(keys=["label"], data_type="tensor"),
                        ]
                    ))

NameError: name 'train_files' is not defined

In [105]:
module = SarcoDataModule(data_dir=Path('data/sarco_final1'), 
do_mixup=False, batch_size=16, 
crop_pixel_width=1, aug_prob=0.2, 
aug_sigma=7, aug_alpha=35)
module.setup()
time_dataloader=module.train_dataloader()
train_ds = module.train_ds

/home/ghita/libs/miniconda3/envs/trains/lib/python3.10/site-packages/pytorch_lightning/core/datamodule.py:95: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/ghita/libs/miniconda3/envs/trains/lib/python3.10/site-packages/pytorch_lightning/core/datamodule.py:114: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/ghita/libs/miniconda3/envs/trains/lib/python3.10/site-packages/pytorch_lightning/core/datamodule.py:88: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/ghita/libs/miniconda3/envs/trains/lib/python3.10/site-packages/pytorch_lightning/core/datamodule.py:107: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_

In [106]:
def benchmark(dl):
    # assert len(time_dataloader) > 10, len(time_dataloader)
    for _ in range(1):
        for (i, _) in enumerate(dl):
            if i > 84:
                break  
        print(i)


In [107]:
%time benchmark(time_dataloader)

ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/ghita/libs/miniconda3/envs/trains/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/ghita/libs/miniconda3/envs/trains/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 40, in fetch
    return self.collate_fn(data)
  File "/home/ghita/libs/miniconda3/envs/trains/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 157, in default_collate
    return elem_type({key: default_collate([d[key] for d in batch]) for key in elem})
  File "/home/ghita/libs/miniconda3/envs/trains/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 157, in <dictcomp>
    return elem_type({key: default_collate([d[key] for d in batch]) for key in elem})
  File "/home/ghita/libs/miniconda3/envs/trains/lib/python3.10/site-packages/torch/utils/data/_utils/collat

In [ ]:
def mixup(data, batch_size, alpha):
    mixed_data=[]
    for k in range(len(data) // batch_size):
        if len(data) < (k + 1) * batch_size:
            batch_data = data[k * batch_size :]
        else:
            batch_data = data[(k * batch_size) : ((k + 1) * batch_size)]
        if alpha > 0:
            lam = np.random.beta(alpha, alpha)
        else:
            lam = 1.0
        index = torch.randperm(batch_size)
        randomized=[batch_data[ind] for ind in index]
        
        for k in range(len(batch_data)):
            l=0
            if batch_data[k].shape==randomized[k].shape:
                mixed_data.append(np.multiply(batch_data[k], lam) + np.multiply(randomized[k],(1 - lam)))
                l+=1
            else :
                mixed_data.append(batch_data[k])
    return mixed_data,l

In [ ]:
ex=[{'input': [1,2,3], 'label':['a','b','c']},{'input': [3,4,5], 'label':['d','e','f']},{'input': [6,7,8], 'label':['g','h','i']} ]
'''aug_ex=[]
for k in range(len(ex)):
    for key in dict(ex):
        aug_ex.append(f"{key}_right":ex[k][key], f"{key}_left":)
list_r=itertools.product(ex,2)
for el in list_r:
    print(el)'''
#combis=list(itertools.combinations(ex,2))
#combis
#dico=dict([(k,[combis[0][0],combis[0][1]]) for k in combis[0][0]])
#print(combis)
#for k in range(len(combis)):
#    combis[k][0]['input_right']=combis[k][0].pop('input')
#    combis[k][1]['label_right']=combis[k][0].pop('label')

#combis
#for k in range(len(combis)):
ex_dataset=monai.data.Dataset(data=ex)
prod=ProductDataset(ex_dataset, ex_dataset)
list(prod)


In [ ]:
def dict_product(dict_dataset:list, length_tuples:int=2):
    """
    This function takes a list of dictionaries dataset and creates another dataset based on the 
    first one but where each key contains not one element but a combination of elements from the original dataset.

    Inputs : 

    dict_dataset : [{'key_a': element_a, 'key_b': element_b}, {...}]

    length_tuples : number of elements wanted for each key. length_tuples=1 -> the function does nothing, length_tuples=2 -> the 
    function creates a cartesian product of the original dataset with no repeated elements. length_tuples=len(dict_dataset) -> all 
    the elements of the dict_dataset are concatenated. 

    Output : 

    Combinations of the input : [{'key_a_0' : element_a_0, 'key_b_0': element_b_0, 
    'key_a_1': element_a_1, 'key_b_1': element_b_1, ..., 'key_a_length_tuples': element_a_length_tuples, 
    'key_b_length_tuples': element_b_length_tuples}, {...}]
    """
    keys=dict(dict_dataset[0])
    output=[]
    combis=list(itertools.combinations(dict_dataset,length_tuples))
    for combi in combis:
        element=[]
        for k in range(length_tuples):
            for key in keys:
                element.append(f"{key}_%s"%k)
                element.append(combi[k][key])
        new_keys=element[0::2]
        values=element[1::2]
        zip_iterator=zip(new_keys, values)
        output.append(dict(zip_iterator))
    return output


In [7]:
ex_batch=monai.data.Dataset(data=train_files[:250])
prod=ProductDataset(ex_batch,ex_batch)
list(prod)[0]
#right, left=list(prod)[1]
#mixing=Mixupd(keys=['input'], alpha=1)
#mixed=mixing(prod)
#plt.imshow(mixed['input'])
#plt.imshow(list(prod)[5][0]['input'])
#print(mixed['input']==list(prod)[5][0]['input'])

[{'input': array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=int16),
  'label': array([[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
  
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
  
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
  
         ...,
  
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
  
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
  
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]]])},
 {'input': array([[0, 0, 0, ..., 0, 0, 0],

In [ ]:
mixing=Mixupd(['input', 'label'], 1)
mixing(ex_batch)

In [ ]:
#mixing=Mixupd(keys=["input", "label"], alpha=1, batch_size=50)
#inputs=[np.reshape(ex_batch[k]["input"],(512,512)) for k in range(len(ex_batch))]
#labels=[ex_batch[k]["label"] for k in range(len(ex_batch))]
#inputs[0], np.multiply(inputs[0], 3)

#a=mixup(inputs,batch_size=5,alpha=0.3)
#a[1]
#type(a[0])
#plt.imshow(a[0][0],cmap='gray')

In [ ]:
#testing the productdatset class



In [5]:
#defining the pl Data Module
class SarcoDataModule(pl.LightningDataModule):
    def __init__(self, data_dir=Path('data/sarco_final1')):
        super().__init__()
        self.data_dir=data_dir
        self.train_transforms=Compose(
                [LoadImaged(keys=["input"], reader='numpyreader', dtype='float32', image_only=True),
                LoadImaged(keys=["label"], reader='numpyreader', dtype='float32', image_only=True),
                SqueezeDimd(keys=['label'],dim=2),
                AddChanneld(keys=["input","label" ]),
                Resized(keys=["input"],spatial_size=(512,512)),
                Resized(keys=["label"], spatial_size=(512,512)),
                #RandRotate90d(keys=["input", "label"], prob=0.5, spatial_axes=[0,1]),
                #Mixupd(keys=["input", "label"], alpha=1, batch_size=20),
                #EnsureTyped(keys=["input"],data_type="tensor",dtype=torch.float),
                #EnsureTyped(keys=['label'],data_type="tensor"),
                ]
            )  
        self.val_transforms=Compose(
                [
                LoadImaged(keys=["input"], reader='numpyreader', dtype='float32', image_only=True),
                LoadImaged(keys=["label"], reader='numpyreader', dtype='float32', image_only=True),
                SqueezeDimd(keys=['label'],dim=2),
                AddChanneld(keys=["input","label" ]),
                Resized(keys=["input"],spatial_size=(512,512)),
                Resized(keys=["label"], spatial_size=(512,512)),
                EnsureTyped(keys=["input"],data_type="tensor",dtype=torch.float),
                EnsureTyped(keys=['label'],data_type="tensor"),
                ]
            )  
        self.predict_transforms=Compose(AddChannel(), EnsureType(data_type='tensor'))
    def setup(self, stage=None):
        if stage=='fit' or stage is None:
            files=[filename for filename in self.data_dir.glob('*') if filename != Path('/home/ghita/milvuetap/ghita_exploration/data/sarco_final1/HyuyQN0Tca')]
            data=[]
            for k in range(len(sorted(files))):
                filename=sorted(files)[k].name.replace('.dcm', '')
                input_dict=self.data_dir/filename/f"{filename}.dcm.npy"
                label_dict=self.data_dir/filename/f"{filename}.nii.npy"
                data.append({"input":input_dict ,"label":label_dict})
            self.train_files, self.val_files=train_test_split(data,test_size=0.2)
            self.val_files, self.test_files=train_test_split(self.val_files, test_size=0.5)
            self.train_ds=monai.data.Dataset(data=self.train_files[:250], transform=self.train_transforms)
            mixing=Mixupd(keys=['input', 'label'], alpha=1)
            self.train_ds=mixing(ProductDataset(self.train_ds, self.train_ds))
            self.train_ds=monai.data.Dataset(data=self.train_ds, transform=EnsureTyped(keys=["input", "label"],data_type="tensor",dtype=torch.float))
            self.val_ds=monai.data.Dataset(data=self.val_files, transform=self.val_transforms)
        if stage=='test':
            self.test_ds=monai.data.Dataset(data=self.test_files, transform=self.val_transforms)
        if stage=='predict':
            to_infer_nifti=nib.load('/home/ghita/milvuetap/ghita_exploration/data/CT.1.2.840.113619.2.416.9243487930095034124053722995226898598.nii')
            infer_slices=[]
            for k in range(to_infer_nifti.get_fdata().shape[2]):
                infer_slices.append(to_infer_nifti.get_fdata()[:,:,k])
            self.pred_ds=monai.data.Dataset(data=infer_slices, transform=self.predict_transforms)
    def train_dataloader(self):
        return(DataLoader(self.train_ds,batch_size=8,num_workers=4))
    def val_dataloader(self):
        return(DataLoader(self.val_ds,batch_size=8,num_workers=4))
    def test_dataloader(self):
        return(DataLoader(self.test_ds, batch_size=8, num_workers=4))
    def predict_dataloader(self):
        return DataLoader(self.pred_ds, batch_size=1) 

In [6]:
#defining the pl module
class pl_UNet(pl.LightningModule):
    def __init__(self, learning_rate):
        super().__init__()
        self.save_hyperparameters()
        self.model=smp.Unet(encoder_name='efficientnet-b0',encoder_weights='imagenet',in_channels=1,classes=1)
        self.learning_rate=learning_rate
    def forward(self,x):
        out=self.model(x)
        return(out) 
    def training_step(self,train_batch,batch_nb):
        input=train_batch['input']
        label=train_batch['label']
        pred=self.forward(input)
        dice=iou_and_dice_pytorch(torch.sigmoid(pred), label)[1]
        loss=monai.losses.DiceLoss(include_background=False, sigmoid=True)
        loss_value=loss(pred,label.float())
        iou_value=iou_and_dice_pytorch(torch.sigmoid(pred),label)[0]
        acc=accuracy(pred,label.type(torch.int))
        prec=average_precision(pred, label)
        logger.log_image(key='prediction', images=[predi for predi in pred])
        logger.log_image(key='input', images=[inp for inp in input])
        logger.log_image(key='label', images=[pr for pr in label])
        self.log('IoU_train', iou_value)
        self.log('loss_train',loss_value)
        self.log('acc_train',acc)
        self.log('Dice_train', dice)
        self.log('Avrg Precision_train', prec)
        return(loss_value)
    def validation_step(self, val_batch,batch_nb):
        input=val_batch['input']
        label=val_batch['label']
        pred=self.forward(input)
        loss=monai.losses.DiceLoss(include_background=False, sigmoid=True)
        loss_value=loss(pred, label.float())
        iou_value=iou_and_dice_pytorch(torch.sigmoid(pred),label)[0]
        dice=iou_and_dice_pytorch(torch.sigmoid(pred), label)[1]
        acc=accuracy(pred,label.type(torch.int))
        prec=average_precision(pred, label)
        logger.log_image(key='prediction_val', images=[predi for predi in pred])
        logger.log_image(key='input_val', images=[inp for inp in input])
        logger.log_image(key='label_val', images=[pr for pr in label])
        self.log('IoU_val',iou_value)
        self.log('loss_val',loss_value)
        acc=accuracy(pred,label.type(torch.int))
        self.log('acc_val',acc)
        self.log('Dice_val', dice)
        self.log('Avrg Precision_ val', prec)
        return(loss_value)
    def test_step(self, test_batch, batch_nb):
        return None
    def predict_step(self, batch, batch_nb):
        x=batch.type(torch.float)
        pred=self(x)
        return(pred)
    def configure_optimizers(self):
        optimizer=torch.optim.Adam(self.parameters(),lr=self.learning_rate)
        return optimizer
    

In [ ]:
#train 
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=pl_UNet(learning_rate=1e-3).to(device)
run = wandb.init(project="train",settings=wandb.Settings(start_method='thread'))
checkpoint_callback_best=ModelCheckpoint(dirpath='/home/ghita/milvuetap/ghita_exploration/checkpoints', monitor='loss_val',save_top_k=1,mode='min')
best_checkpoint_path=checkpoint_callback_best.best_model_path
checkpoint_callback_last=ModelCheckpoint(dirpath='/home/ghita/milvuetap/ghita_exploration/checkpoints', monitor='step',save_top_k=1,mode='max')
last_checkpoint_path=checkpoint_callback_last.best_model_path
logger=WandbLogger(project='train')
trainer=pl.Trainer(logger=logger,max_epochs=1,auto_lr_find=True,auto_scale_batch_size=True, callbacks=[checkpoint_callback_best, checkpoint_callback_last], accelerator='gpu', gpus=1)
trainer.fit(model, datamodule=SarcoDataModule(data_dir=Path('data/sarco_final1'), augment=True))

In [4]:
run=wandb.init(project="train", settings=wandb.Settings(start_method='fork'))


In [2]:
#checking whether the inverse functions work 

example_dict=[{"input": Path("/home/ghita/milvuetap/ghita_exploration/data/sarco_final1/0bzRUjziqI/0bzRUjziqI.dcm.npy"), "label": Path("/home/ghita/milvuetap/ghita_exploration/data/sarco_final1/0bzRUjziqI/0bzRUjziqI.nii.npy")}]
transforms=Compose([LoadImaged(keys=["input", "label"], reader="numpyreader",
                    dtype="float32",
                    image_only=True),
                    SqueezeDimd(keys=["label"], dim=2),
                    Cropd(keys=["input", "label"], pixel_width=1, pixel_height=1),
                    AddChanneld(keys=["input", "label"]), 
                    #Resized(keys=["input", "label"], spatial_size=(512, 512))
                    ])

dataset=monai.data.Dataset(data=example_dict, transform=transforms)

In [11]:
post_transforms=SqueezeDimd(keys=['input', 'label'], dim=0), Invertd(
                    keys=["input"],
                    transform=transforms,
                    orig_keys=["input"],
                 )

In [12]:
testing_inverse=monai.data.Dataset(data=dataset, transform=post_transforms)


In [15]:
list(testing_inverse)[0]["input"]

RuntimeError: applying transform (<monai.transforms.utility.dictionary.SqueezeDimd object at 0x7f9f18d083a0>, <monai.transforms.post.dictionary.Invertd object at 0x7f9f18a57070>)

In [4]:
#converting to nii 



d2n.convert_directory('/home/ghita/milvuetap/ghita_exploration/data/sarco dataset 1 & 2 improved/0bzRUjziqI.dcm', '/home/ghita/milvuetap/ghita_exploration/data/example.nii')

In [3]:
nib.load('/home/ghita/milvuetap/ghita_exploration/data/sarco dataset 1 & 2 improved/0bzRUjziqI.dcm.nii')

FileNotFoundError: No such file or no access: '/home/ghita/milvuetap/ghita_exploration/data/sarco dataset 1 & 2 improved/0bzRUjziqI.dcm.nii'